In [1]:
!pip install torch torchvision
!pip install transformers==2.2.0
!pip install seqeval
!pip install tensorboardx
!pip install simpletransformers==0.9.1

Looking in indexes: https://pypi.org/simple, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.snapshots.pypi/simple/, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.releases.pypi/simple/

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.5.0 requires torch==1.4.0, but you'll have torch 1.7.1 which is incompatible.



  Using cached torchvision-0.5.0-cp37-cp37m-win_amd64.whl (1.2 MB)
Looking in indexes: https://pypi.org/simple, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.snapshots.pypi/simple/, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.releases.pypi/simple/
  Using cached transformers-2.2.0-py3-none-any.whl (360 kB)
Processing c:\users\i345144\appdata\local\pip\cache\wheels\69\09\d1\bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0\sacremoses-0.0.43-py3-none-any.whl
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorlayer 2.2.2 requires cloudpickle>=0.8.1, which is not installed.
tensorlayer 2.2.2 requires imageio>=2.5.0, which is not installed.
tensorlayer 2.2.2 requires progressbar2>=3.39.3, which is not installed.
tensorlayer 2.2.2 requires scikit-image>=0.15.0, which is not installed.


Looking in indexes: https://pypi.org/simple, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.snapshots.pypi/simple/, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.releases.pypi/simple/
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16176 sha256=536e3463ba39612af7631ffb10ad894a48df8be15d1474d721edd186dde1d252
  Stored in directory: c:\users\i345144\appdata\local\pip\cache\wheels\05\96\ee\7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.snapshots.pypi/simple/, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.releases.pypi/simple/
Looking in indexes: https://pypi.org/simple, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.snapshots.pypi/simple/, http://nexus.wdf.sap.corp:8081/nexus/content/groups/build.releases.pypi/simple/


In [10]:
## Import required libraries
%matplotlib inline

import pandas as pd
import numpy as np

import gc
import requests
import os
import io
#from google.colab import drive 
import datetime
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score
from sklearn import preprocessing 

import torch
print("Cuda available" if torch.cuda.is_available() is True else "CPU")
print("PyTorch version: ", torch.__version__)

CPU
PyTorch version:  1.7.1+cpu


In [13]:
 
## dataset-2
#drive.mount('/content/gdrive')

train=pd.read_csv('./Dataset_3/FNID-dataset/dataset/fake_news_detection(LIAR)/liar_train.csv')
test=pd.read_csv('./Dataset_3/FNID-dataset/dataset/fake_news_detection(LIAR)/liar_test.csv')
val=pd.read_csv('./Dataset_3/FNID-dataset/dataset/fake_news_detection(LIAR)/liar_dev.csv')

print(train.head())



      id                       date  \
0  18178  2020-03-18T13:26:42-04:00   
1   3350  2011-03-04T09:12:59-05:00   
2  14343  2017-07-21T11:52:44-04:00   
3  15579  2018-07-10T15:52:26-04:00   
4   3690  2011-05-04T16:31:24-04:00   

                                       speaker  \
0                              Instagram posts   
1                                   Glenn Beck   
2                                   Mike Pence   
3                                     Bloggers   
4  National Republican Congressional Committee   

                                           statement  \
0         "COVID-19 started because we eat animals."   
1  Says Michelle Obama has 43 people on her staff...   
2  Says President Donald Trump "has signed more l...   
3  "US representatives promise implement of UN gu...   
4  "The federal government borrows $4 billion eve...   

                                             sources  \
0  ['https://www.cdc.gov/coronavirus/2019-ncov/ca...   
1  ['http://www

C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
train_df = train[['statement', 'label-liar']]
eval_df = test[['statement', 'label-liar']]

train_df['statement'] = train_df['statement'].str.lower()
eval_df['statement'] = eval_df['statement'].str.lower()

train_df=train_df[train_df['label-liar'].isin(['true','false'])]
eval_df=eval_df[eval_df['label-liar'].isin(['true','false'])]

label_encoder = preprocessing.LabelEncoder() 
train_df['label-liar']= label_encoder.fit_transform(train_df['label-liar'])
eval_df['label-liar']= label_encoder.fit_transform(eval_df['label-liar']) 

print(train_df.head())
print(len(train_df))
print(len(eval_df))

                                            statement  label-liar
7   "the united states is the only industrialized ...           1
8   "hillary's main extracurricular activity in la...           0
9     says ohio gov. john kasich is "very unpopular."           0
14  "if we make no changes over the next 10 years,...           1
17  "while mayor kenney personally doesn't support...           0
5330
457


C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
print(train['label-liar'].unique())

['barely-true' 'pants-fire' 'half-true' 'mostly-true' 'true' 'false']


In [25]:
print(len(train_df))

15052


In [18]:
print(len(train_df[train_df['label-liar'].isin(['mostly-true'])]))

2631


In [19]:
print(len(train_df[train_df['label-liar'].isin(['pants-fire'])]))

1775


In [20]:
print(len(train_df[train_df['label-liar'].isin(['barely-true'])]))

2483


In [21]:
print(len(train_df[train_df['label-liar'].isin(['half-true'])]))

2833


In [22]:
print(len(train_df[train_df['label-liar'].isin(['true'])]))

2050


In [23]:
print(len(train_df[train_df['label-liar'].isin(['false'])]))

3280


In [26]:
df=train_df[train_df['label-liar'].isin(['true','false'])]
print(df.head())
print(len(df))

                                            statement label-liar
7   "the united states is the only industrialized ...       true
8   "hillary's main extracurricular activity in la...      false
9     says ohio gov. john kasich is "very unpopular."      false
14  "if we make no changes over the next 10 years,...       true
17  "while mayor kenney personally doesn't support...      false
5330


In [29]:

args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',

   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 50,
   'eval_batch_size': 50,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 1,
   'weight_decay': 0,
   'learning_rate': 1e-3,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,

   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,

   'overwrite_output_dir': True,
   'reprocess_input_data': True,
}

start = datetime.datetime.now()

model = ClassificationModel('xlnet', 'xlnet-base-cased', args = args, use_cuda=False)

model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result, model_outputs)



Converting to features started.


Running loss: 0.776913

C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\transformers\optimization.py:146: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Running loss: 0.686638

C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\torch\optim\lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.788402

Training of xlnet model complete. Saved to outputs/.
Converting to features started.


C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


NameError: name 'outputs' is not defined

In [32]:
eval_df['BERT_prediction'] = np.argmax(model_outputs, axis = 1)

del model
del result
del model_outputs
del wrong_predictions
gc.collect()
torch.cuda.empty_cache()

end = datetime.datetime.now()
exec_time=end-start
print("time to train the model : \n", exec_time)


time to train the model : 
 7:54:44.673298


In [33]:
def report_results(A, B):
    A_name = A.name
    B_name = B.name
    
    df = pd.DataFrame({'A':A,
                       'B':B})
    df = df.dropna()
    A = df['A']
    B = df['B']
    
    acc = accuracy_score(B, A)
    f1 = f1_score(B, A)
    prec = precision_score(B, A)
    rec = recall_score(B, A)
    ROC = roc_auc_score(B, A)
    
    print('Candidate: '+A_name+' | Ground Truth: '+B_name+'\n')
    print('accuracy: %0.2f \nprecision: %0.2f \nrecall: %0.2f \nF1 score: %0.2f \nROC AUC: %0.2f \n' % (acc, prec, rec, f1, ROC))
    

In [37]:
report_results( eval_df['BERT_prediction'],eval_df['label-liar'])

Candidate: BERT_prediction | Ground Truth: label-liar

accuracy: 0.54 
precision: 0.00 
recall: 0.00 
F1 score: 0.00 
ROC AUC: 0.50 



C:\Users\i345144\AppData\Local\Continuum\anaconda3\envs\FakeNewsV2\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
